In [1]:
import pandas as pd
import pymysql

In [2]:
# 获得沪深300成分股列表
import jqdatasdk as jq
jq.auth('USERNAME', 'PASSWORD')

df_stocks = pd.DataFrame(jq.get_index_stocks('000300.XSHG'))

auth success 


In [3]:
# 更改后缀为万得代码格式
df_stocks.columns = ['stockID']
df_stocks['stockCode'] = df_stocks['stockID'].str.split('.').apply(lambda x:x[0])
df_stocks['stockSuffix'] = df_stocks['stockID'].str.split('.').apply(lambda x:x[1])

df_stocks.replace('XSHE', 'SZ', inplace=True)
df_stocks.replace('XSHG', 'SH', inplace=True)

df_stocks['stockID'] = df_stocks['stockCode']+'.'+df_stocks['stockSuffix']

stock_list = df_stocks['stockID'].unique().tolist()

In [6]:
# 读取数据
stocks = tuple(stock_list)
conw = pymysql.connect(host='', user="", password="", database="wind")
cursorw = conw.cursor()
cursorw.execute(
    f'SELECT S_INFO_WINDCODE, TRADE_DT, S_VAL_PE, S_VAL_PB_NEW, S_VAL_PS, S_DQ_TURN, S_DQ_MV, NET_ASSETS_TODAY \
        FROM ASHAREEODDERIVATIVEINDICATOR \
            where S_INFO_WINDCODE in {stocks}\
                and TRADE_DT between 20170101 and 20220701')
df = pd.DataFrame(cursorw.fetchall())
df.columns = ['S_INFO_WINDCODE', 'TRADE_DT', 'S_VAL_PE', 'S_VAL_PB_NEW', 'S_VAL_PS', 'S_DQ_TURN', 'S_DQ_MV', 'NET_ASSETS_TODAY']

# 保存数据为gzip格式
df.to_csv('value_factor.gz', compression='gzip', index=False)

# 读取数据方法
# df = pd.read_csv('value_factor.gz')

In [7]:
df = pd.read_csv('value_factor.gz')
df.head()

,S_INFO_WINDCODE,TRADE_DT,S_VAL_PE,S_VAL_PB_NEW,S_VAL_PS,S_DQ_TURN,S_DQ_MV,NET_ASSETS_TODAY
0,000725.SZ,20170101,61.4432,1.3033,2.0677,NaN,6.872158e+06,7.714270e+10
1,600346.SH,20170101,NaN,4.4023,27.9980,NaN,5.562664e+05,5.346761e+09
2,600010.SH,20170101,NaN,1.9186,4.0373,NaN,4.392025e+06,4.734914e+10
3,600585.SH,20170101,11.9574,1.2163,1.7631,NaN,6.783496e+06,7.389383e+10
4,600383.SH,20170101,18.2782,1.8064,1.7871,NaN,5.833101e+06,3.238384e+10
